# 🦌 Compounds 🧪

```python

```

In [ ]:
from collections import defaultdict
from dataclasses import dataclass, field
from inspect import isclass
from typing import ClassVar, Dict, List, Optional, Tuple, Union
from uuid import uuid4

import importnb

# import ipyelk.diagram.elk_export
import ipywidgets as W
import networkx as nx
from IPython.display import JSON, SVG, display

import ipyelk
import ipyelk.nx
import ipyelk.tools
import ipyelk.tools.tools

# from ipyelk.contrib.library import logic_gates as logic
from ipyelk import Elk
from ipyelk.contrib.elements import Compound, Label, Node, Port, Registry
from ipyelk.contrib.library import logic_gates
from ipyelk.contrib.shapes import connectors as conn
from ipyelk.contrib.shapes import shapes
from ipyelk.diagram import elk_export, elk_model
from ipyelk.diagram import layout_options as opt
from ipyelk.diagram import symbol
from ipyelk.diagram.defs import ConnectorDef, Def
from ipyelk.diagram.elk_model import strip_none
from ipyelk.diagram.layout_options.layout import ELKRectanglePacking
from ipyelk.diagram.symbol import Symbol
from ipyelk.transform import merge

### TODO

- how transformer deals with layoutOptions / properties / cssClasses
  - probably not great to have them in too many places...
  - "inherited" from higher nodes in the graph vs explicitly set in the node data
- label shapes
  - icon view
- sizing for record compounds
- composing compounds

In [ ]:
with Registry():
    n1 = Node(ports={"a": Port()}, labels=[Label(text="hello")])
    n1.a._parent
    n1.add_child(Node())

    data = n1.to_json()

JSON(data)

In [ ]:
n1 = Node(labels=[Label(text="hello", labels=[Label(text="subtext")])])
# n1.a = Port()

n1.add_child(Node())
n2 = Node()
n2.a = Port()
ilk = Compound()
ilk.connect(n2.a, n1)

In [ ]:
with Registry():
    display(JSON(n1.to_json()))

In [ ]:
# configure app
app = Elk(
    transformer=ipyelk.nx.XELK(
        #         layouts={
        #             elk_model.ElkRoot: {
        #                 "parents": diagram_opts,
        #             },
        #         },
        source=ilk.source(),
    ),
    layout={"height": "100%"},
    style={
        " .hidden": {
            #             "display": "none",
        }
    },
)
app

In [ ]:
ng = Node(shape=logic_gates.Nand_Gate())
n1.add_child(ng)

app.diagram.defs = {logic_gates.Nand_Gate.identifier: logic_gates.Nand_Gate.shape}
app.transformer.source = ilk.source()
from ipyelk.contrib.elements.compound import nx_wrap

nx_ng = nx_wrap(ng, ilk.registry)

In [ ]:
ng.shape = logic_gates.Nand_Gate()
app.transformer.source = ilk.source()

In [ ]:
app.transformer.get_properties(nx_ng)

In [ ]:
JSON(app.transformer.value)

In [ ]:
from IPython.display import JSON

JSON([d for *_, d in app.transformer.source[0].edges(data=True)])

In [ ]:
JSON(app.transformer.value)

In [ ]:
JSON([d for _, d in ilk.source()[0].nodes(data=True)])

In [ ]:
logic_gates.Nand_Gate.shape

In [ ]:
g, tree = ilk.source(context=Registry())

In [ ]:
from IPython.display import JSON

JSON([d for _, d in ilk.source()[0].nodes(data=True)])

In [ ]:
from IPython.display import JSON

JSON([d for *_, d in ilk.source()[0].edges(data=True)])

# Experiments for Record Nodes

In [ ]:
app = Elk(
    transformer=ipyelk.nx.XELK(
        #         layouts={
        #             elk_model.ElkRoot: {
        #                 "parents": diagram_opts,
        #             },
        #         },
        #         source=(g, tree),
    ),
    layout={"height": "100%"},
    style={
        " .hidden": {
            #             "display": "none",
        }
    },
)
app

In [ ]:
from dataclasses import dataclass
from typing import Dict


@dataclass
class Record(symbol.Symbol):
    def get_layoutOptions(self) -> Dict:
        return opt.OptionsWidget(
            options=[
                opt.HierarchyHandling(),
                opt.Padding(left=0, right=0, bottom=0, top=0),
                opt.NodeSpacing(spacing=0),
                opt.EdgeNodeSpacing(spacing=0),
                opt.AspectRatio(),
                opt.ExpandNodes(activate=True),
                opt.NodeLabelPlacement(horizontal="center", vertical="center"),
                #         opt.NodeSizeConstraints(),
                opt.ComponentsSpacing(spacing=0),
                opt.NodeSpacing(spacing=0),
            ]
        ).value


# .to_json()
g = nx.Graph()
tree = nx.DiGraph()
r = Record()
g.add_node("a", **r.to_json(id="a"))

In [ ]:
g = nx.Graph()
tree = nx.DiGraph()

diagram_opts = opt.OptionsWidget(
    options=[
        opt.LayoutAlgorithm(),
        opt.HierarchyHandling(),
        #         opt.NodeSpacing(spacing=0),
    ]
).value
record_layout = opt.OptionsWidget(
    options=[
        #         opt.LayoutAlgorithm(value=ELKRectanglePacking.identifier),
        opt.HierarchyHandling(),
        opt.Padding(left=0, right=0, bottom=0, top=0),
        opt.NodeSpacing(spacing=0),
        opt.EdgeNodeSpacing(spacing=0),
        opt.AspectRatio(),
        opt.ExpandNodes(activate=True),
        opt.NodeLabelPlacement(horizontal="center", vertical="center"),
        #         opt.NodeSizeConstraints(),
        opt.ComponentsSpacing(spacing=0),
        opt.NodeSpacing(spacing=0),
    ]
).value

compartment_opts = opt.OptionsWidget(
    options=[
        opt.NodeLabelPlacement(horizontal="center", vertical="center"),
        #         opt.NodeSizeConstraints(),
        opt.NodeSpacing(spacing=0),
    ]
).value

g.add_node("record", labels=[], layoutOptions=record_layout)

compartments = ["a", "b"]
for i, c in enumerate(compartments):
    g.add_node(
        c, width=40, height=36, layoutOptions=compartment_opts, ports=[c + str(i)]
    )
    tree.add_edge("record", c)

# g.add_node("n1")

# configure app
app = Elk(
    transformer=ipyelk.nx.XELK(
        layouts={
            elk_model.ElkRoot: {
                "parents": diagram_opts,
            },
        },
        source=(g, tree),
    ),
    layout={"height": "100%"},
    style={
        " .hidden": {
            #             "display": "none",
        }
    },
)
app

In [ ]:
app.diagram.mark_layout

In [ ]:
g = nx.Graph()
tree = nx.DiGraph()

record_layout = opt.OptionsWidget(
    options=[
        opt.LayoutAlgorithm(value=ELKRectanglePacking.identifier),
        opt.HierarchyHandling(),
        opt.Padding(left=0, right=0, bottom=0, top=0),
        opt.NodeSpacing(spacing=0),
        opt.EdgeNodeSpacing(spacing=0),
        opt.AspectRatio(),
        opt.ExpandNodes(activate=True),
    ]
).value

compartment_opts = opt.OptionsWidget(
    options=[
        opt.NodeLabelPlacement(horizontal="center", vertical="center"),
        opt.NodeSizeConstraints(),
    ]
).value

compartments = ["a", "b"]
for i, c in enumerate(compartments):
    g.add_node(
        c,
        width=40 + 2 * i,
        height=30,
        layoutOptions=compartment_opts,
        #         ports=[c + str(i)]
    )


# configure app
app = Elk(
    transformer=ipyelk.nx.XELK(
        layouts={
            elk_model.ElkRoot: {
                "parents": record_layout,
            },
        },
        source=(g, tree),
    ),
    layout={"height": "100%"},
    style={
        " .hidden": {
            #             "display": "none",
        }
    },
)
app